# Read bronze fxdata table and convert to fact and dimension tables in silver layer


In [6]:
%%sql
-- you will notice there's an issue with the currency pair in this data
SELECT * FROM PERS.bronze_fxdata WHERE to_date(cast(unix_timestamp(`YYYY/MM/DD`, 'yyyy/MM/dd') as timestamp)) = '2024-01-02' LIMIT 10;
-- basically, base currency and quote currency from the PERS FX dataset is flipped in the bronze data

StatementMeta(, e7438441-5135-4846-9035-3971e706ed4d, 11, Finished, Available)

<Spark SQL result set with 10 rows and 5 fields>

###### Create temporary views with data transformation logic
Notice that base currency and quote currency is flipped in the view to accommodate for data issues

In [9]:
%%sql
CREATE VIEW IF NOT EXISTS vwcurrency AS
SELECT 1 as col;

-- create temp view vwcurrency.  this will be used to populate currency dimension
ALTER VIEW vwcurrency AS
WITH currencycte AS (
SELECT DISTINCT split(ISO4217, '/')[0] AS `currency_id`
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
UNION 
SELECT DISTINCT split(ISO4217, '/')[1] AS `currency_id`
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
)
SELECT DISTINCT `currency_id` FROM currencycte;

CREATE VIEW IF NOT EXISTS vwexchangerate AS
SELECT 1 as col;

-- create temp view vwexchangerate.  this will be used to populate exchange rate fact table
ALTER VIEW vwexchangerate AS
SELECT     
    split(ISO4217, '/')[1] AS `base_currency_id`, 
    split(ISO4217, '/')[0] AS `quote_currency_id`,
    to_date(cast(unix_timestamp(`YYYY/MM/DD`, 'yyyy/MM/dd') as timestamp)) AS `exchange_date`,
    Data AS `exchange_rate` 
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
AND Data IS NOT NULL
AND `YYYY/MM/DD` IS NOT NULL;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
%%sql
-- check view results
SELECT * FROM vwcurrency;
SELECT * FROM vwexchangerate LIMIT 10;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 11 rows and 1 fields>

<Spark SQL result set with 10 rows and 4 fields>

###### Create silver fact and dimension tables if not exists

In [11]:
%%sql
-- create currency table
CREATE TABLE IF NOT EXISTS PERS.silver_currency
(
    currency_id string,
    currency_name string,
    create_dt date,
    modify_dt date
) USING delta;

-- create exchange_rate table
CREATE TABLE IF NOT EXISTS PERS.silver_exchange_rate
(
    base_currency_id string,
    quote_currency_id string,
    exchange_date date,
    exchange_rate double
) USING delta; 

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

###### Insert into currency dimension table

In [12]:
%%sql
MERGE INTO PERS.`silver_currency`
USING vwcurrency
ON PERS.`silver_currency`.`currency_id` = vwcurrency.`currency_id`
WHEN NOT MATCHED THEN
INSERT (PERS.`silver_currency`.`currency_id`,PERS.`silver_currency`.`currency_name`,PERS.`silver_currency`.`create_dt`,PERS.`silver_currency`.`modify_dt`)
VALUES (vwcurrency.`currency_id`,vwcurrency.`currency_id`,current_timestamp(),current_timestamp());

StatementMeta(, e7438441-5135-4846-9035-3971e706ed4d, 26, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

###### Insert into exchange rate fact table

In [13]:
%%sql
MERGE INTO PERS.silver_exchange_rate TGT
USING vwexchangerate SRC
ON (TGT.base_currency_id = SRC.base_currency_id AND TGT.quote_currency_id = SRC.quote_currency_id AND TGT.exchange_date = CAST(SRC.exchange_date AS DATE))
WHEN MATCHED AND TGT.exchange_rate <> SRC.exchange_rate THEN
    UPDATE SET TGT.exchange_rate = SRC.exchange_rate
WHEN NOT MATCHED THEN
    INSERT (base_currency_id, quote_currency_id, exchange_date, exchange_rate) VALUES (SRC.base_currency_id, SRC.quote_currency_id, SRC.exchange_date, SRC.exchange_rate);

StatementMeta(, e7438441-5135-4846-9035-3971e706ed4d, 27, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

Validate table results

In [14]:
%%sql 
-- validate results in silver tables
SELECT * FROM PERS.silver_currency;
SELECT * FROM PERS.silver_exchange_rate WHERE exchange_date IS NOT NULL LIMIT 10;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 11 rows and 4 fields>

<Spark SQL result set with 10 rows and 4 fields>